In [ ]:
import os # for path mainly
import sys # for path mainly
import logging # for status and log output
import matplotlib.pyplot as plt # for plotting charts
import seaborn as sns # for plotting default nice charts

# Ensure path resolves relative to notebook file (robust in different CWDs)
notebook_dir = (
    os.path.dirname(os.path.abspath(__file__)) if "__file__" in globals() else os.getcwd()
)
sys.path.append(os.path.normpath(os.path.join(notebook_dir, "..", "pylib")))


from handle_sqlite import save_dataframe_to_db, read_table_as_dataframe

In [ ]:
# diagnostic + fix (run in a notebook cell)
import os, sys
notebook_dir = (os.path.dirname(os.path.abspath(__file__)) if "__file__" in globals() else os.getcwd())
print("CWD:", os.getcwd())
print("notebook_dir:", notebook_dir)

pylib_path = os.path.normpath(os.path.join(notebook_dir, "..", "pylib"))
db_path = os.path.normpath(os.path.join(notebook_dir, "..", "data_output", "dwh_data.db"))
print("pylib exists?:", os.path.exists(pylib_path), pylib_path)
print("db exists?:", os.path.exists(db_path), db_path)

# Option A: set CWD to project root so relative paths work
project_root = os.path.normpath(os.path.join(notebook_dir, ".."))
os.chdir(project_root)
print("new CWD:", os.getcwd())

# Option B (recommended): use absolute paths when calling helpers:
# metadata = read_table_as_dataframe("newspapers", db_path)

---
load data from dhw

In [ ]:
# load from dwh the newspaper informations per date and their usage of klima
metadata = read_table_as_dataframe("newspapers", "data_output/dwh_data.db")
metadata.head(2)

In [ ]:
# load from dwh the found klima words
context = read_table_as_dataframe("context", "data_output/dwh_data.db")
context.head(2)

---
cast types can help analysis and performance

In [ ]:
context.info()

In [ ]:
context = context.astype({'pre_context': 'string',
                         'post_context': 'string',
                         'prefix': 'string',
                         'suffix': 'string',})
context.info()

In [ ]:
metadata[metadata.newspaper_name == 'heise']

In [ ]:
# dublicated newspapers published
metadata.duplicated().sum()

In [ ]:
# dublicates of context, including newspaper id. indicates static use
# of klima, like navigation term of a paper.
context.duplicated().sum()

In [ ]:
# % of news pages (not newspaper-companys) with at least once
# klima in actual data
context.newspaper_id.nunique() / len(context) * 100

% of newspaper-companys, that never mentioned klima

In [ ]:
#metadata.groupby(['newspaper_name'])['klima_mentions_count'].sum()
klima_counts_per_company = metadata.pivot_table(values='klima_mentions_count', index='newspaper_name', aggfunc='sum')

In [ ]:
total_companys = len(klima_counts_per_company)
total_companys

In [ ]:
no_klima_companies = (klima_counts_per_company['klima_mentions_count'] == 0).sum()
no_klima_companies

In [ ]:
no_klima_companies / total_companys * 100

In [ ]:
dict(klima_counts_per_company_sorted)

In [ ]:
# histogram of the times, klima is used in one single paper publish
ax = sns.countplot(data=metadata, x="klima_mentions_count", palette="flare")
#ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
# look at the lower freq edges

top_three = sorted(metadata['klima_mentions_count'].unique())[:5]
counts = metadata['klima_mentions_count'].value_counts().loc[top_three]

ax = sns.barplot(y=counts.index, x=counts.values, orient='h', palette="flare")
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
# look at the higher freq edges and its newspapers

top_three = sorted(metadata['klima_mentions_count'].unique())[-5:]
counts = metadata['klima_mentions_count'].value_counts().loc[top_three]
counts

In [ ]:
ax=sns.countplot(data=context, x="newspaper_id", order=context.newspaper_id.value_counts().iloc[:5].index, palette="flare")
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
ax=sns.countplot(data=context, x="newspaper_id", order=context.newspaper_id.value_counts().iloc[:15].index, palette="flare")
#ax.bar_label(ax.containers[0])
plt.show()

## Merging for deeper eda


In [ ]:
# make sure the one to many relationship meets expectations

In [ ]:
# Count the number of rows for each newspaper_id in the metadata and context tables
metadata_counts = metadata['newspaper_id'].value_counts()

# Check if there are any newspaper_ids that appear more than once in the metadata table
problematic_metadata = metadata_counts[metadata_counts > 1]

print(f"Problematic newspaper_id in metadata (appears more than once):\n{problematic_metadata}")

# Check for any cases where there are no rows in metadata for context's newspaper_id
missing_metadata = context[~context['newspaper_id'].isin(metadata['newspaper_id'])]
print(f"Context rows with missing metadata:\n{missing_metadata}")

In [ ]:
context[~context['newspaper_id'].isin(metadata['newspaper_id'])]

In [ ]:
# Checking the first few rows of both dataframes
print(metadata.head(2))
print(context.head(2))

In [ ]:
import pandas as pd

In [ ]:
# Check for one-to-many relationship based on 'newspaper_id'
merged = pd.merge(context, metadata, on="newspaper_id", how="inner")

# Checking the number of unique newspaper_id in both tables
print(f"Unique newspaper_id in metadata: {metadata['newspaper_id'].nunique()}")
print(f"Unique newspaper_id in context: {context['newspaper_id'].nunique()}")
print(f"Unique newspaper_id in merged: {merged['newspaper_id'].nunique()}")

In [ ]:
merged

---

Null analysis

In [ ]:
dates = pd.to_datetime(merged['data_published'], errors='coerce')
dates.min(), dates.max()

In [ ]:
# First, convert data_published to datetime if not already done
merged['data_published'] = pd.to_datetime(merged['data_published'], errors='coerce')

# Get min/max dates per newspaper to understand coverage
coverage = merged.groupby('newspaper_name')['data_published'].agg(['min', 'max', 'count']).reset_index()
coverage.columns = ['newspaper_name', 'first_date', 'last_date', 'total_records']
coverage = coverage.sort_values('first_date')

print(coverage)


In [ ]:
import matplotlib.dates as mdates

# Ensure datetime
metadata['data_published'] = pd.to_datetime(metadata['data_published'], errors='coerce')
metadata['has_klima'] = metadata['klima_mentions_count'] > 0

# Sort newspapers by first appearance
newspapers = (
    metadata.groupby('newspaper_name')['data_published'].min()
    .sort_values()
    .index.tolist()
)

date_min = metadata['data_published'].min()
date_max = metadata['data_published'].max()
all_dates = pd.date_range(start=date_min, end=date_max, freq='D')

fig, ax = plt.subplots(figsize=(16, 12))
y_pos = 0
for np_name in newspapers:
    np_meta = metadata[metadata['newspaper_name'] == np_name]
    crawled_dates = set(np_meta['data_published'].dt.date)
    klima_dates = set(np_meta[np_meta['has_klima']]['data_published'].dt.date)

    for d in all_dates:
        if d.date() in klima_dates:
            ax.barh(y_pos, 1, left=d, height=0.8, color='steelblue', edgecolor='none')
        elif d.date() in crawled_dates:
            ax.barh(y_pos, 1, left=d, height=0.8, color='lightgray', edgecolor='none')
    y_pos += 1

ax.set_yticks(range(len(newspapers)))
ax.set_yticklabels(newspapers, fontsize=9)
ax.set_xlabel('Datum Veröffentlicht', fontsize=11, fontweight='bold')
ax.set_ylabel('Newspaper Name', fontsize=11, fontweight='bold')
ax.set_title('Gecrawlte Daten: Blau=Klima, Grau=Ohne Klima', fontsize=14, fontweight='bold', pad=20)

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=4))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b. %y'))
plt.xticks(rotation=45, ha='right')
ax.set_xlim(date_min, date_max)
ax.grid(axis='x', alpha=0.3, linestyle=':')

# Marker for crawler change on 01. April 2022
crawler_change = pd.Timestamp('2022-04-21')
ax.axvline(crawler_change, color='crimson', linestyle='--', linewidth=1.2, alpha=0.9, label='Crawler-Änderung 01.04.2022')

# Optional legend
ax.legend(loc='upper right', fontsize=9)

plt.tight_layout()
plt.show()